Seurat Wrapper + Circos visualization 
================ 
Robin Browaeys 
18-1-2021 
 
<!-- github markdown built using  
rmarkdown::render("vignettes/seurat_wrapper_circos.Rmd", output_format = "github_document") 
--> 
 
In this vignette, you can learn how to perform a basic NicheNet analysis 
on a Seurat v3 object - and how to visualize the output in a circos 
plot. This vignette demonstrates the same workflow as shown in [Perform 
NicheNet analysis starting from a Seurat 
object](seurat_wrapper.md):`vignette("seurat_wrapper", package="nichenetr")`, 
but adds a circos plot visualization as shown in [Circos plot 
visualization to show active ligand-target links between interacting 
cells](circos.md):`vignette("circos", package="nichenetr")`. For more 
detailed information about the NicheNet workflow, check those vignettes. 
This vignette was made upon popular request to demonstrate how those two 
vignettes can be combined into one analysis workflow. Note that we as 
developers of NicheNet generally recommend a visualization of the output 
by combining several heatmaps (ligand activity, ligand-target links, 
ligand-receptor links, ligand expression, ligand LFC,…) over using a 
circos plot visualization. Certainly for cases with many sender cell 
types and ligands that are expressed by more than one sender cell type. 
Because in those cases, the circos plot is much less informative and 
could lead to wrong interpretation of the results. 
 
As example expression data of interacting cells, we will use mouse 
NICHE-seq data from Medaglia et al. to explore intercellular 
communication in the T cell area in the inguinal lymph node before and 
72 hours after lymphocytic choriomeningitis virus (LCMV) infection \[See 
@medaglia\_spatial\_2017\]. We will NicheNet to explore immune cell 
crosstalk in response to this LCMV infection. 
 
In this dataset, differential expression is observed between CD8 T cells 
in steady-state and CD8 T cells after LCMV infection. NicheNet can be 
applied to look at how several immune cell populations in the lymph node 
(i.e., monocytes, dendritic cells, NK cells, B cells, CD4 T cells) can 
regulate and induce these observed gene expression changes. NicheNet 
will specifically prioritize ligands from these immune cells and their 
target genes that change in expression upon LCMV infection. 
 
The used NicheNet networks, ligand-target matrix and example expression 
data of interacting cells can be downloaded from Zenodo. The NicheNet 
networks and ligand-target matrix at 
[![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.3260758.svg)](https://doi.org/10.5281/zenodo.3260758) 
and the Seurat object of the processed NICHE-seq single-cell data at 
[![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.3531889.svg)](https://doi.org/10.5281/zenodo.3531889). 
 
# Prepare NicheNet analysis 
 
## Load required packages, read in the Seurat object with processed expression data of interacting cells and NicheNet’s ligand-target prior model, ligand-receptor network and weighted integrated networks. 
 
### Load Packages: 
 


In [ ]:
library(nichenetr) 
library(Seurat) # Please update to Seurat v4 
library(tidyverse) 
library(circlize) 


 
If you would use and load other packages, we recommend to load these 3 
packages after the others. 
 
### Read in NicheNet’s ligand-target prior model, ligand-receptor network and weighted integrated networks: 
 


In [ ]:
ligand_target_matrix = readRDS(url("https://zenodo.org/record/3260758/files/ligand_target_matrix.rds")) 
ligand_target_matrix[1:5,1:5] # target genes in rows, ligands in columns 
##                 CXCL1        CXCL2        CXCL3        CXCL5         PPBP 
## A1BG     3.534343e-04 4.041324e-04 3.729920e-04 3.080640e-04 2.628388e-04 
## A1BG-AS1 1.650894e-04 1.509213e-04 1.583594e-04 1.317253e-04 1.231819e-04 
## A1CF     5.787175e-04 4.596295e-04 3.895907e-04 3.293275e-04 3.211944e-04 
## A2M      6.027058e-04 5.996617e-04 5.164365e-04 4.517236e-04 4.590521e-04 
## A2M-AS1  8.898724e-05 8.243341e-05 7.484018e-05 4.912514e-05 5.120439e-05 
 
lr_network = readRDS(url("https://zenodo.org/record/3260758/files/lr_network.rds")) 
head(lr_network) 
## # A tibble: 6 x 4 
##   from  to    source         database 
##   <chr> <chr> <chr>          <chr>    
## 1 CXCL1 CXCR2 kegg_cytokines kegg     
## 2 CXCL2 CXCR2 kegg_cytokines kegg     
## 3 CXCL3 CXCR2 kegg_cytokines kegg     
## 4 CXCL5 CXCR2 kegg_cytokines kegg     
## 5 PPBP  CXCR2 kegg_cytokines kegg     
## 6 CXCL6 CXCR2 kegg_cytokines kegg 
 
weighted_networks = readRDS(url("https://zenodo.org/record/3260758/files/weighted_networks.rds")) 
head(weighted_networks$lr_sig) # interactions and their weights in the ligand-receptor + signaling network 
## # A tibble: 6 x 3 
##   from  to     weight 
##   <chr> <chr>   <dbl> 
## 1 A1BG  ABCC6  0.422  
## 2 A1BG  ACE2   0.101  
## 3 A1BG  ADAM10 0.0970 
## 4 A1BG  AGO1   0.0525 
## 5 A1BG  AKT1   0.0855 
## 6 A1BG  ANXA7  0.457 
head(weighted_networks$gr) # interactions and their weights in the gene regulatory network 
## # A tibble: 6 x 3 
##   from  to     weight 
##   <chr> <chr>   <dbl> 
## 1 A1BG  A2M    0.0294 
## 2 AAAS  GFAP   0.0290 
## 3 AADAC CYP3A4 0.0422 
## 4 AADAC IRF8   0.0275 
## 5 AATF  ATM    0.0330 
## 6 AATF  ATR    0.0355 


 
### Read in the expression data of interacting cells: 
 


In [ ]:
seuratObj = readRDS(url("https://zenodo.org/record/3531889/files/seuratObj.rds")) 
seuratObj@meta.data %>% head() 
##         nGene nUMI orig.ident aggregate res.0.6 celltype nCount_RNA nFeature_RNA 
## W380370   880 1611      LN_SS        SS       1    CD8 T       1607          876 
## W380372   541  891      LN_SS        SS       0    CD4 T        885          536 
## W380374   742 1229      LN_SS        SS       0    CD4 T       1223          737 
## W380378   847 1546      LN_SS        SS       1    CD8 T       1537          838 
## W380379   839 1606      LN_SS        SS       0    CD4 T       1603          836 
## W380381   517  844      LN_SS        SS       0    CD4 T        840          513 


 
Visualize which cell populations are present: CD4 T cells (including 
regulatory T cells), CD8 T cells, B cells, NK cells, dendritic cells 
(DCs) and inflammatory monocytes 
 


In [ ]:
seuratObj@meta.data$celltype %>% table() # note that the number of cells of some cell types is very low and should preferably be higher for a real application 
## . 
##     B CD4 T CD8 T    DC  Mono    NK  Treg  
##   382  2562  1645    18    90   131   199 
DimPlot(seuratObj, reduction = "tsne") 


 
![](seurat_wrapper_circos_files/figure-gfm/unnamed-chunk-4-1.png)<!-- --> 
 


In [ ]:
seuratObj@meta.data$aggregate %>% table() 
## . 
## LCMV   SS  
## 3886 1141 
DimPlot(seuratObj, reduction = "tsne", group.by = "aggregate") 


 
![](seurat_wrapper_circos_files/figure-gfm/unnamed-chunk-5-1.png)<!-- --> 
 
# Perform the NicheNet analysis 
 
## NicheNet analysis on Seurat object: explain differential expression between two conditions 
 
In this case study, the receiver cell population is the ‘CD8 T’ cell 
population, whereas the sender cell populations are ‘CD4 T’, ‘Treg’, 
‘Mono’, ‘NK’, ‘B’ and ‘DC’. The above described functions will consider 
a gene to be expressed when it is expressed in at least a predefined 
fraction of cells in one cluster (default: 10%). 
 
The gene set of interest are the genes differentially expressed in CD8 T 
cells after LCMV infection. The condition of interest is thus ‘LCMV’, 
whereas the reference/steady-state condition is ‘SS’. The notion of 
conditions can be extracted from the metadata column ‘aggregate’, the 
method to calculate the differential expression is the standard Seurat 
Wilcoxon test. 
 
The number of top-ranked ligands that are further used to predict active 
target genes and construct an active ligand-receptor network is 20 by 
default. 
 
To perform the NicheNet analysis with these specifications, run the 
following: 
 


In [ ]:
# indicated cell types should be cell class identities 
# check via:  
# seuratObj %>% Idents() %>% table() 
sender_celltypes = c("CD4 T","Treg", "Mono", "NK", "B", "DC") 
nichenet_output = nichenet_seuratobj_aggregate( 
  seurat_obj = seuratObj,  
  receiver = "CD8 T",  
  condition_colname = "aggregate", condition_oi = "LCMV", condition_reference = "SS",  
  sender = sender_celltypes,  
  ligand_target_matrix = ligand_target_matrix, lr_network = lr_network, weighted_networks = weighted_networks, organism = "mouse") 
## [1] "Read in and process NicheNet's networks" 
## [1] "Define expressed ligands and receptors in receiver and sender cells" 
## [1] "Perform DE analysis in receiver cell" 
## [1] "Perform NicheNet ligand activity analysis" 
## [1] "Infer active target genes of the prioritized ligands" 
## [1] "Infer receptors of the prioritized ligands" 


 
### Interpret the NicheNet analysis output 
 
#### Ligand activity analysis results 
 
A first thing NicheNet does, is prioritizing ligands based on predicted 
ligand activity. To see the ranking of these ligands, run the following 
command: 
 


In [ ]:
nichenet_output$ligand_activities 
## # A tibble: 44 x 6 
##    test_ligand auroc  aupr pearson  rank bona_fide_ligand 
##    <chr>       <dbl> <dbl>   <dbl> <dbl> <lgl>            
##  1 Ebi3        0.638 0.234  0.197      1 FALSE            
##  2 Il15        0.582 0.163  0.0961     2 TRUE             
##  3 Crlf2       0.549 0.163  0.0758     3 FALSE            
##  4 App         0.499 0.141  0.0655     4 TRUE             
##  5 Tgfb1       0.494 0.140  0.0558     5 TRUE             
##  6 Ptprc       0.536 0.149  0.0554     6 TRUE             
##  7 H2-M3       0.525 0.157  0.0528     7 TRUE             
##  8 Icam1       0.543 0.142  0.0486     8 TRUE             
##  9 Cxcl10      0.531 0.141  0.0408     9 TRUE             
## 10 Adam17      0.517 0.137  0.0359    10 TRUE             
## # ... with 34 more rows 


 
These ligands are expressed by one or more of the input sender cells. To 
see which cell population expresses which of these top-ranked ligands, 
you can run the following: 
 


In [ ]:
DotPlot(seuratObj, features = nichenet_output$top_ligands %>% rev(), cols = "RdYlBu") + RotatedAxis() 



 
![](seurat_wrapper_circos_files/figure-gfm/unnamed-chunk-8-1.png)<!-- --> 
 
As you can see, most op the top-ranked ligands seem to be mainly 
expressed by dendritic cells and monocytes. 
 
It could also be interesting to see whether some of these ligands are 
differentially expressed after LCMV infection. 
 


In [ ]:
DotPlot(seuratObj, features = nichenet_output$top_ligands %>% rev(), split.by = "aggregate") + RotatedAxis() 



 
![](seurat_wrapper_circos_files/figure-gfm/unnamed-chunk-9-1.png)<!-- --> 
 


In [ ]:
VlnPlot(seuratObj, features = c("Il15", "Cxcl10","Cxcl16"), split.by = "aggregate", pt.size = 0, combine = FALSE) 
## [[1]] 


 
![](seurat_wrapper_circos_files/figure-gfm/unnamed-chunk-10-1.png)<!-- --> 
 
    ##  
    ## [[2]] 
 
![](seurat_wrapper_circos_files/figure-gfm/unnamed-chunk-10-2.png)<!-- --> 
 
    ##  
    ## [[3]] 
 
![](seurat_wrapper_circos_files/figure-gfm/unnamed-chunk-10-3.png)<!-- --> 
 
#### Inferred active ligand-target links 
 
NicheNet also infers active target genes of these top-ranked ligands. To 
see which top-ranked ligands are predicted to have regulated the 
expression of which differentially expressed genes, you can run 
following command for a heatmap visualization: 
 


In [ ]:
nichenet_output$ligand_target_heatmap 



 
![](seurat_wrapper_circos_files/figure-gfm/unnamed-chunk-11-1.png)<!-- --> 
 
## Circos plots to visualize ligand-target and ligand-receptor interactions 
 
This visualization groups the top predicted active ligands according to 
the strongest expressing cell type. Therefore we need to determine per 
cell type which ligands they express more strongly than the other cell 
types. 
 
### Calculate average ligand expression in sender cells 
 


In [ ]:
# avg_expression_ligands = AverageExpression(seuratObj %>% subset(subset = aggregate == "LCMV"),features = nichenet_output$top_ligands) # if want to look specifically in LCMV-only cells 
avg_expression_ligands = AverageExpression(seuratObj, features = nichenet_output$top_ligands) 


 
### Assign ligands to sender cells 
 
To assign ligands to sender cell type, we can e.g. look for which sender 
cell types show an expression that is higher than the average + SD. 
 


In [ ]:
sender_ligand_assignment = avg_expression_ligands$RNA %>% apply(1, function(ligand_expression){ 
  ligand_expression > (ligand_expression %>% mean() + ligand_expression %>% sd()) 
  }) %>% t() 
sender_ligand_assignment = sender_ligand_assignment %>% apply(2, function(x){x[x == TRUE]}) %>% purrr::keep(function(x){length(x) > 0}) 
names(sender_ligand_assignment) 
## [1] "B"    "NK"   "Mono" "DC" 


 
The top ligands seem to be most strongly expressed by B cells, NK cells, 
monocytes and DCs. We will know also look at which ligands are common 
across multiple cell types (= those that are specific to &gt; 1 cell 
type, or those that were not assigned to a cell type in the previous 
block of code) 
 
Determine now which prioritized ligands are expressed by CAFs and or 
endothelial cells 
 


In [ ]:
all_assigned_ligands = sender_ligand_assignment %>% lapply(function(x){names(x)}) %>% unlist() 
unique_ligands = all_assigned_ligands %>% table() %>% .[. == 1] %>% names() 
general_ligands = nichenet_output$top_ligands %>% setdiff(unique_ligands) 
 
B_specific_ligands = sender_ligand_assignment$B %>% names() %>% setdiff(general_ligands) 
NK_specific_ligands = sender_ligand_assignment$NK %>% names() %>% setdiff(general_ligands) 
Mono_specific_ligands = sender_ligand_assignment$Mono %>% names() %>% setdiff(general_ligands) 
DC_specific_ligands = sender_ligand_assignment$DC %>% names() %>% setdiff(general_ligands) 
 
ligand_type_indication_df = tibble( 
  ligand_type = c(rep("B-specific", times = B_specific_ligands %>% length()), 
                  rep("NK-specific", times = NK_specific_ligands %>% length()), 
                  rep("Mono-specific", times = Mono_specific_ligands %>% length()), 
                  rep("DC-specific", times = DC_specific_ligands %>% length()), 
                  rep("General", times = general_ligands %>% length())), 
  ligand = c(B_specific_ligands, NK_specific_ligands, Mono_specific_ligands, DC_specific_ligands, general_ligands)) 


 
### Define the ligand-target links of interest 
 
To avoid making a circos plots with too many ligand-target links, we 
will show only links with a weight higher than a predefined cutoff: 
links belonging to the 40% of lowest scores were removed. Not that this 
cutoffs and other cutoffs used for this visualization can be changed 
according to the user’s needs. 
 


In [ ]:
active_ligand_target_links_df = nichenet_output$ligand_target_df %>% mutate(target_type = "LCMV-DE") %>% inner_join(ligand_type_indication_df) # if you want ot make circos plots for multiple gene sets, combine the different data frames and differentiate which target belongs to which gene set via the target type 
 
cutoff_include_all_ligands = active_ligand_target_links_df$weight %>% quantile(0.40) 
 
active_ligand_target_links_df_circos = active_ligand_target_links_df %>% filter(weight > cutoff_include_all_ligands) 
 
ligands_to_remove = setdiff(active_ligand_target_links_df$ligand %>% unique(), active_ligand_target_links_df_circos$ligand %>% unique()) 
targets_to_remove = setdiff(active_ligand_target_links_df$target %>% unique(), active_ligand_target_links_df_circos$target %>% unique()) 
   
circos_links = active_ligand_target_links_df %>% filter(!target %in% targets_to_remove &!ligand %in% ligands_to_remove) 


 
Prepare the circos visualization: give each segment of ligands and 
targets a specific color and order 
 


In [ ]:
grid_col_ligand =c("General" = "lawngreen", 
            "NK-specific" = "royalblue", 
            "B-specific" = "darkgreen", 
            "Mono-specific" = "violet", 
            "DC-specific" = "steelblue2") 
grid_col_target =c( 
            "LCMV-DE" = "tomato") 
 
grid_col_tbl_ligand = tibble(ligand_type = grid_col_ligand %>% names(), color_ligand_type = grid_col_ligand) 
grid_col_tbl_target = tibble(target_type = grid_col_target %>% names(), color_target_type = grid_col_target) 
 
circos_links = circos_links %>% mutate(ligand = paste(ligand," ")) # extra space: make a difference between a gene as ligand and a gene as target! 
circos_links = circos_links %>% inner_join(grid_col_tbl_ligand) %>% inner_join(grid_col_tbl_target) 
links_circle = circos_links %>% select(ligand,target, weight) 
 
ligand_color = circos_links %>% distinct(ligand,color_ligand_type) 
grid_ligand_color = ligand_color$color_ligand_type %>% set_names(ligand_color$ligand) 
target_color = circos_links %>% distinct(target,color_target_type) 
grid_target_color = target_color$color_target_type %>% set_names(target_color$target) 
 
grid_col =c(grid_ligand_color,grid_target_color) 
 
# give the option that links in the circos plot will be transparant ~ ligand-target potential score 
transparency = circos_links %>% mutate(weight =(weight-min(weight))/(max(weight)-min(weight))) %>% mutate(transparency = 1-weight) %>% .$transparency  


 
Prepare the circos visualization: order ligands and targets 
 


In [ ]:
target_order = circos_links$target %>% unique() 
ligand_order = c(Mono_specific_ligands, DC_specific_ligands, NK_specific_ligands,B_specific_ligands, general_ligands) %>% c(paste(.," ")) %>% intersect(circos_links$ligand) 
order = c(ligand_order,target_order) 


 
Prepare the circos visualization: define the gaps between the different 
segments 
 


In [ ]:
width_same_cell_same_ligand_type = 0.5 
width_different_cell = 6 
width_ligand_target = 15 
width_same_cell_same_target_type = 0.5 
 
gaps = c( 
  # width_ligand_target, 
  rep(width_same_cell_same_ligand_type, times = (circos_links %>% filter(ligand_type == "Mono-specific") %>% distinct(ligand) %>% nrow() -1)), 
  width_different_cell, 
  rep(width_same_cell_same_ligand_type, times = (circos_links %>% filter(ligand_type == "DC-specific") %>% distinct(ligand) %>% nrow() -1)), 
  width_different_cell, 
  rep(width_same_cell_same_ligand_type, times = (circos_links %>% filter(ligand_type == "NK-specific") %>% distinct(ligand) %>% nrow() -1)), 
  width_different_cell, 
  rep(width_same_cell_same_ligand_type, times = (circos_links %>% filter(ligand_type == "B-specific") %>% distinct(ligand) %>% nrow() -1)), 
  width_different_cell, 
  rep(width_same_cell_same_ligand_type, times = (circos_links %>% filter(ligand_type == "General") %>% distinct(ligand) %>% nrow() -1)), 
  width_ligand_target, 
  rep(width_same_cell_same_target_type, times = (circos_links %>% filter(target_type == "LCMV-DE") %>% distinct(target) %>% nrow() -1)), 
  width_ligand_target 
  ) 


 
Render the circos plot (all links same transparancy). Only the widths of 
the blocks that indicate each target gene is proportional the 
ligand-target regulatory potential (\~prior knowledge supporting the 
regulatory interaction). 
 


In [ ]:
circos.par(gap.degree = gaps) 
chordDiagram(links_circle, directional = 1,order=order,link.sort = TRUE, link.decreasing = FALSE, grid.col = grid_col,transparency = 0, diffHeight = 0.005, direction.type = c("diffHeight", "arrows"),link.arr.type = "big.arrow", link.visible = links_circle$weight >= cutoff_include_all_ligands,annotationTrack = "grid",  
    preAllocateTracks = list(track.height = 0.075)) 
# we go back to the first track and customize sector labels 
circos.track(track.index = 1, panel.fun = function(x, y) { 
    circos.text(CELL_META$xcenter, CELL_META$ylim[1], CELL_META$sector.index, 
        facing = "clockwise", niceFacing = TRUE, adj = c(0, 0.55), cex = 1) 
}, bg.border = NA) # 


 
![](seurat_wrapper_circos_files/figure-gfm/unnamed-chunk-19-1.png)<!-- --> 
 


In [ ]:
circos.clear() 



 
Render the circos plot (degree of transparancy determined by the 
regulatory potential value of a ligand-target interaction) 
 


In [ ]:
circos.par(gap.degree = gaps) 
chordDiagram(links_circle, directional = 1,order=order,link.sort = TRUE, link.decreasing = FALSE, grid.col = grid_col,transparency = transparency, diffHeight = 0.005, direction.type = c("diffHeight", "arrows"),link.arr.type = "big.arrow", link.visible = links_circle$weight >= cutoff_include_all_ligands,annotationTrack = "grid",  
    preAllocateTracks = list(track.height = 0.075)) 
# we go back to the first track and customize sector labels 
circos.track(track.index = 1, panel.fun = function(x, y) { 
    circos.text(CELL_META$xcenter, CELL_META$ylim[1], CELL_META$sector.index, 
        facing = "clockwise", niceFacing = TRUE, adj = c(0, 0.55), cex = 1) 
}, bg.border = NA) # 


 
![](seurat_wrapper_circos_files/figure-gfm/unnamed-chunk-20-1.png)<!-- --> 
 


In [ ]:
circos.clear() 



 
Save circos plot to an svg file 
 


In [ ]:
svg("ligand_target_circos.svg", width = 10, height = 10) 
circos.par(gap.degree = gaps) 
chordDiagram(links_circle, directional = 1,order=order,link.sort = TRUE, link.decreasing = FALSE, grid.col = grid_col,transparency = transparency, diffHeight = 0.005, direction.type = c("diffHeight", "arrows"),link.arr.type = "big.arrow", link.visible = links_circle$weight >= cutoff_include_all_ligands,annotationTrack = "grid", 
    preAllocateTracks = list(track.height = 0.075)) 
# we go back to the first track and customize sector labels 
circos.track(track.index = 1, panel.fun = function(x, y) { 
    circos.text(CELL_META$xcenter, CELL_META$ylim[1], CELL_META$sector.index, 
        facing = "clockwise", niceFacing = TRUE, adj = c(0, 0.55), cex = 1) 
}, bg.border = NA) # 
circos.clear() 
dev.off() 
## png  
##   2 


 
### Visualize ligand-receptor interactions of the prioritized ligands in a circos plot 
 


In [ ]:
lr_network_top_df = nichenet_output$ligand_receptor_df %>% mutate(receptor_type = "LCMV_CD8T_receptor") %>% inner_join(ligand_type_indication_df) 



In [ ]:
grid_col_ligand =c("General" = "lawngreen", 
            "NK-specific" = "royalblue", 
            "B-specific" = "darkgreen", 
            "Mono-specific" = "violet", 
            "DC-specific" = "steelblue2") 
grid_col_receptor =c( 
            "LCMV_CD8T_receptor" = "darkred") 
 
grid_col_tbl_ligand = tibble(ligand_type = grid_col_ligand %>% names(), color_ligand_type = grid_col_ligand) 
grid_col_tbl_receptor = tibble(receptor_type = grid_col_receptor %>% names(), color_receptor_type = grid_col_receptor) 
 
circos_links = lr_network_top_df %>% mutate(ligand = paste(ligand," ")) # extra space: make a difference between a gene as ligand and a gene as receptor! 
circos_links = circos_links %>% inner_join(grid_col_tbl_ligand) %>% inner_join(grid_col_tbl_receptor) 
links_circle = circos_links %>% select(ligand,receptor, weight) 
 
ligand_color = circos_links %>% distinct(ligand,color_ligand_type) 
grid_ligand_color = ligand_color$color_ligand_type %>% set_names(ligand_color$ligand) 
receptor_color = circos_links %>% distinct(receptor,color_receptor_type) 
grid_receptor_color = receptor_color$color_receptor_type %>% set_names(receptor_color$receptor) 
 
grid_col =c(grid_ligand_color,grid_receptor_color) 
 
# give the option that links in the circos plot will be transparant ~ ligand-receptor potential score 
transparency = circos_links %>% mutate(weight =(weight-min(weight))/(max(weight)-min(weight))) %>% mutate(transparency = 1-weight) %>% .$transparency  


 
Prepare the circos visualization: order ligands and receptors 
 


In [ ]:
receptor_order = circos_links$receptor %>% unique() 
ligand_order = c(Mono_specific_ligands, DC_specific_ligands, NK_specific_ligands,B_specific_ligands, general_ligands) %>% c(paste(.," ")) %>% intersect(circos_links$ligand) 
order = c(ligand_order,receptor_order) 


 
Prepare the circos visualization: define the gaps between the different 
segments 
 


In [ ]:
width_same_cell_same_ligand_type = 0.5 
width_different_cell = 6 
width_ligand_receptor = 15 
width_same_cell_same_receptor_type = 0.5 
 
gaps = c( 
  # width_ligand_target, 
  rep(width_same_cell_same_ligand_type, times = (circos_links %>% filter(ligand_type == "Mono-specific") %>% distinct(ligand) %>% nrow() -1)), 
  width_different_cell, 
  rep(width_same_cell_same_ligand_type, times = (circos_links %>% filter(ligand_type == "DC-specific") %>% distinct(ligand) %>% nrow() -1)), 
  width_different_cell, 
  rep(width_same_cell_same_ligand_type, times = (circos_links %>% filter(ligand_type == "NK-specific") %>% distinct(ligand) %>% nrow() -1)), 
  width_different_cell, 
  rep(width_same_cell_same_ligand_type, times = (circos_links %>% filter(ligand_type == "B-specific") %>% distinct(ligand) %>% nrow() -1)), 
  width_different_cell, 
  rep(width_same_cell_same_ligand_type, times = (circos_links %>% filter(ligand_type == "General") %>% distinct(ligand) %>% nrow() -1)), 
  width_ligand_receptor, 
  rep(width_same_cell_same_receptor_type, times = (circos_links %>% filter(receptor_type == "LCMV_CD8T_receptor") %>% distinct(receptor) %>% nrow() -1)), 
  width_ligand_receptor 
  ) 


 
Render the circos plot (all links same transparancy). Only the widths of 
the blocks that indicate each receptor is proportional the 
ligand-receptor interaction weight (\~prior knowledge supporting the 
interaction). 
 


In [ ]:
circos.par(gap.degree = gaps) 
chordDiagram(links_circle, directional = 1,order=order,link.sort = TRUE, link.decreasing = FALSE, grid.col = grid_col,transparency = 0, diffHeight = 0.005, direction.type = c("diffHeight", "arrows"),link.arr.type = "big.arrow", link.visible = links_circle$weight >= cutoff_include_all_ligands,annotationTrack = "grid",  
    preAllocateTracks = list(track.height = 0.075)) 
# we go back to the first track and customize sector labels 
circos.track(track.index = 1, panel.fun = function(x, y) { 
    circos.text(CELL_META$xcenter, CELL_META$ylim[1], CELL_META$sector.index, 
        facing = "clockwise", niceFacing = TRUE, adj = c(0, 0.55), cex = 0.8) 
}, bg.border = NA) # 


 
![](seurat_wrapper_circos_files/figure-gfm/unnamed-chunk-26-1.png)<!-- --> 
 


In [ ]:
circos.clear() 



 
Render the circos plot (degree of transparancy determined by the prior 
interaction weight of the ligand-receptor interaction - just as the 
widths of the blocks indicating each receptor) 
 


In [ ]:
circos.par(gap.degree = gaps) 
chordDiagram(links_circle, directional = 1,order=order,link.sort = TRUE, link.decreasing = FALSE, grid.col = grid_col,transparency = transparency, diffHeight = 0.005, direction.type = c("diffHeight", "arrows"),link.arr.type = "big.arrow", link.visible = links_circle$weight >= cutoff_include_all_ligands,annotationTrack = "grid",  
    preAllocateTracks = list(track.height = 0.075)) 
# we go back to the first track and customize sector labels 
circos.track(track.index = 1, panel.fun = function(x, y) { 
    circos.text(CELL_META$xcenter, CELL_META$ylim[1], CELL_META$sector.index, 
        facing = "clockwise", niceFacing = TRUE, adj = c(0, 0.55), cex = 0.8) 
}, bg.border = NA) # 


 
![](seurat_wrapper_circos_files/figure-gfm/unnamed-chunk-27-1.png)<!-- --> 
 


In [ ]:
circos.clear() 



 
Save circos plot to an svg file 
 


In [ ]:
svg("ligand_receptor_circos.svg", width = 15, height = 15) 
circos.par(gap.degree = gaps) 
chordDiagram(links_circle, directional = 1,order=order,link.sort = TRUE, link.decreasing = FALSE, grid.col = grid_col,transparency = transparency, diffHeight = 0.005, direction.type = c("diffHeight", "arrows"),link.arr.type = "big.arrow", link.visible = links_circle$weight >= cutoff_include_all_ligands,annotationTrack = "grid", 
    preAllocateTracks = list(track.height = 0.075)) 
# we go back to the first track and customize sector labels 
circos.track(track.index = 1, panel.fun = function(x, y) { 
    circos.text(CELL_META$xcenter, CELL_META$ylim[1], CELL_META$sector.index, 
        facing = "clockwise", niceFacing = TRUE, adj = c(0, 0.55), cex = 0.8) 
}, bg.border = NA) # 
circos.clear() 
dev.off() 
## png  
##   2 
